In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!gcloud config set project lively-tensor-284509
!gsutil -m cp -r gs://nlp_recsys2020/recsys2020_data/train.parquet/ /content/
!gsutil -m cp -r gs://nlp_recsys2020/recsys2020_data/train_userembeddings.parquet/ /content/

In [11]:
!gsutil -m cp -r gs://nlp_recsys2020/recsys2020_data/test.parquet/ /content/
!gsutil -m cp -r gs://nlp_recsys2020/recsys2020_data/test_userembeddings.parquet/ /content/

Copying gs://nlp_recsys2020/recsys2020_data/test.parquet/_common_metadata...
Copying gs://nlp_recsys2020/recsys2020_data/test.parquet/_metadata...
Copying gs://nlp_recsys2020/recsys2020_data/test.parquet/part.0.parquet...
Copying gs://nlp_recsys2020/recsys2020_data/test.parquet/part.1.parquet...
Copying gs://nlp_recsys2020/recsys2020_data/test.parquet/part.10.parquet...
Copying gs://nlp_recsys2020/recsys2020_data/test.parquet/part.100.parquet...
Copying gs://nlp_recsys2020/recsys2020_data/test.parquet/part.101.parquet...
Copying gs://nlp_recsys2020/recsys2020_data/test.parquet/part.102.parquet...
Copying gs://nlp_recsys2020/recsys2020_data/test.parquet/part.103.parquet...
Copying gs://nlp_recsys2020/recsys2020_data/test.parquet/part.104.parquet...
Copying gs://nlp_recsys2020/recsys2020_data/test.parquet/part.105.parquet...
Copying gs://nlp_recsys2020/recsys2020_data/test.parquet/part.106.parquet...
Copying gs://nlp_recsys2020/recsys2020_data/test.parquet/part.107.parquet...
Copying gs:

In [ ]:
!pip install "dask[complete]" "dask-ml[complete]" numpy pandas pyarrow tqdm partd

In [ ]:
import os

from dask.distributed import Client
from dask_ml.xgboost import XGBClassifier
import numpy as np
from pathlib import Path
from functools import partial
import tqdm
import shutil
import pandas as pd
import dask
import dask.array as da
import dask.dataframe as dd
import math
pd.set_option('max_colwidth', 0)

ds_name = 'train'
data_dir = Path('/content')
train_tweets = data_dir / f"{ds_name}.parquet/"
train_userembeddings = data_dir / f"{ds_name}_userembeddings.parquet/"

In [ ]:
df_train_tweets = dd.read_parquet(str(train_tweets))

In [ ]:
df_train_userembeddings = dd.read_parquet(str(train_userembeddings))

In [ ]:
df_train_tweets = df_train_tweets.set_index('user_id', sorted=True)

In [ ]:
# df_train_userembeddings = df_train_userembeddings.set_index('user_id')

In [ ]:
df = df_train_tweets.merge(df_train_userembeddings, left_index=True, right_index=True, suffixes=(None, "_u"))

In [ ]:
print(df.index)
print(df.columns)
print(df.dtypes)
print(df.head(10))

Dask Index Structure:
npartitions=260
000088971D76A0C34F5D5D4246E0E433    object
0109542FAB11DEC785DB9EFFEAB39EE6    ...   
                                   ...    
FEF7DD651EE399C89819562200FCE900    ...   
FFFF0597BF4FE93C5C8288BD0DAB9EF2    ...   
Name: user_id, dtype: object
Dask Name: join-indexed, 2081 tasks
Index(['tweet_id', 'tweet_type', 'tokens', 'author_follower_count',
       'author_following_count', 'author_is_verified', 'user_follower_count',
       'user_following_count', 'user_is_verified', 'follows',
       ...
       'e758_u', 'e759_u', 'e760_u', 'e761_u', 'e762_u', 'e763_u', 'e764_u',
       'e765_u', 'e766_u', 'e767_u'],
      dtype='object', length=1561)
tweet_id                  object 
tweet_type                object 
tokens                    object 
author_follower_count     int32  
author_following_count    int32  
                          ...    
e763_u                    float64
e764_u                    float64
e765_u                    float64
e766_u 

In [ ]:
# df = df.persist()

In [2]:
df_pandas = df.compute()
df_pandas.to_parquet(str(data_dir / 'training_set.parquet'))

NameError: ignored

In [1]:
train_cols = ['e{:03d}'.format(i) for i in range(768)] \
    + ['e{:03d}_u'.format(i) for i in range(768)] \
    + [
        "author_follower_count",
        "author_following_count",
        "user_follower_count",
        "user_following_count",
        "num_hashtags",
        "num_media",
        "num_links",
        "num_domains",
        "author_is_verified",
        "user_is_verified",
        "follows",
    ]
classes = ['retweet', 'reply', 'like', 'retweet_with_comment']
label_cols = ['has_' + c for c in classes]

In [3]:
import os

from dask.distributed import Client
from dask_ml.xgboost import XGBClassifier
import numpy as np
from pathlib import Path
from functools import partial
import tqdm
import shutil
import pandas as pd
import dask
import dask.array as da
import dask.dataframe as dd
import math
pd.set_option('max_colwidth', 0)

ds_name = 'train'
data_dir = Path('/content')
train_tweets = data_dir / f"{ds_name}.parquet/"
train_userembeddings = data_dir / f"{ds_name}_userembeddings.parquet/"

In [4]:
df_pandas = pd.read_parquet(str(data_dir / 'training_set.parquet'))

In [5]:
df_pandas = df_pandas.sample(frac=0.1)

In [6]:
df_pandas[train_cols].values

array([[-0.7111215591430664, -2.5657432079315186, -0.9480071663856506,
        ..., False, False, False],
       [-1.419252634048462, -1.0212411880493164, -1.5002517700195312,
        ..., False, False, False],
       [0.3841181695461273, -0.35378071665763855, 0.4715070128440857,
        ..., False, False, False],
       ...,
       [0.8025840520858765, -0.004410469904541969, 0.6346856355667114,
        ..., False, False, False],
       [-0.276525616645813, -1.8811094760894775, -1.3457859754562378,
        ..., False, False, False],
       [1.3800463676452637, -0.19588974118232727, 0.03404612839221954,
        ..., False, False, True]], dtype=object)

In [7]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
# now you can import normally from ensemble
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

from joblib import dump, load

In [10]:
clf = RandomForestClassifier(random_state=1, verbose=1)
# clf.fit(df_pandas[train_cols].values, df_pandas[label_cols[0]].values)  
multi_clf = MultiOutputClassifier(clf, n_jobs=-1)
multi_clf.fit(df_pandas[train_cols].values, df_pandas[label_cols].values)  
dump(multi_clf, str(data_dir / f'model_sklearn_multi_rf_10perc')) 

['/content/model_sklearn_multi_rf_10perc']